# ML 프로젝트
- 평가지표 : RMSLE
  + 차이(실제값 - 예측값) / N
  + 차이의 제곱, 차이의 절댓값, 차이 로그변환, 등
  + 공통 원리 : 평가지표의 값은 낮으면 낮을수록 좋은 모델
- 예시
  + 모델 1 평가지표 : 100
  + 모델 2 평가지표 : 50
  => 좋은 모델 : 모델 2
- Kaggle 주제 : 칼로리 소모량(Calories)에 미치는 영향을 탐색적으로 분석해서 칼로리 소모량을 예측하는게 point!
- 칼로리 소모량에 가장 유의미한 변수 추출 및 그것을 가지고 예측

# 데이터 소개
- 신체 정보 : 성별, 나이, 키, 몸무게
- 운동 특성 : 운동 시간, 심박수, 체온
- 타겟 : 칼로리 소모량
- 독립변수 : 신체 정보 + 운동 특성
- 종속변수 : 칼로리 소모량

# 환경 설정
- 깔끔한 시각화를 위한 환경 설정

In [1]:
import matplotlib.pyplot as plt 
# 한글출력
plt.rcParams['font.family'] = 'Malgun Gothic' #  Windows 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

# 데이터 가져오기

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [3]:
test = pd.read_csv('test.csv')
test.head()
# test에는 train가 다르게 Calories 열이 없음

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


## 머신러닝의 원리
- 데이터 + 해답 으로 머신러닝 알고리즘 적용하여 규칙 찾기
- 데이터 : 신체정보 + 운동특성
- 해답 : 칼로리 소모량
- 프로젝트를 진행하면, 대부분의 데이터는 데이터만 있음 / 해답은 존재하지 않음
  + 우리가 해야할 것은, 해답을 정의를 해야함

# 프로세스
- 데이터 수집
- 데이터 가공 & 탐색적 데이터 분석
- 데이터셋 분리
- 모델링
- 평가지표
- 최종모델 선정
- test.csv 데이터에 적용
- submission 파일로 내보내기      # 웹서비스 구현
- Kaggle에 업로드

## 데이터 가공(전처리) & 탐색적 데이터 분석

In [4]:
# 결측치 확인
train.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [5]:
# 탐색적 데이터 분석 (eda.ipynb 참조)
print('eda.ipynb 확인')

eda.ipynb 확인


In [6]:
# 열 값 확인
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories'],
      dtype='object')

In [7]:
# 이상치 탐색
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_feature_boxplots(df):
    train2 = train.copy()
    if 'id' in train2.columns:
        train2 = train2.drop(columns=['id'])
    if 'Calories' in train2.columns:
        train2 = train2.drop(columns=['Calories'])

    # 숫자형 컬럼만 선택 (박스플롯에 적합)
    num_cols = train2.select_dtypes(include=np.number).columns

    # 그리드 크기 계산
    n_cols = 3
    n_rows = int(np.ceil(len(num_cols) / n_cols))

    # 박스플롯 그리기
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
    axes = axes.flatten()  # 1D 배열로

    for i, col in enumerate(num_cols):
        sns.boxplot(y=df[col], ax=axes[i])
        axes[i].set_title(f'{col}', fontsize=12)

    # 나머지 subplot 비우기
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    fig.suptitle('Boxplots of Features (excluding Calories and id)', fontsize=16)
    plt.tight_layout()
    plt.show()

# Feature Engineering
- 관련 추천 서적 (https://www.yes24.com/product/goods/67472798)
- 수치 데이터를 처리하는 방식 (Min-Max 정규화, Z-score 표준화)
  + 핵심은 수치 데이터마다 단위가 다 다르다는 것
  + ex) Age vs Body_Temp
    - Age == 39와 Body_Temp == 39는 다른 의미
- 범주(문자) 데이터를 처리하는 방식 (Sex : male, female)
  + One-Hot Encoding (이 데이터에서는 이것만 진행)
  + Ordinal Encoding (서열 척도 : 등급 => 확실하게 서열이 맺어진 것에만 사용 ex) 수능등급)
- if, 시도(강원도 ~ 제주도) 분석
  + 사회과학 관점에서는 시도 구분은 One-Hot Encoding 진행
  + 경제학 관점 : Ordinal Encoding (because, 경제 지표로 봤을 때, 시도 서열로 구분 가능)

## 인코딩 변환

In [8]:
# Sex 컬럼을 One-Hot Encoding, Pandas Methond 존재
sex_encoded = pd.get_dummies(train['Sex'], prefix='Sex')
train = pd.concat([train, sex_encoded], axis = 1)
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Sex_female,Sex_male
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,False,True
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,True,False
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,True,False
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,False,True
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,True,False


In [9]:
# Sex 열을 제거 -> 인코딩했으니까 필요 X
train = train.drop('Sex', axis=1)
train.head()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Sex_female,Sex_male
0,0,36,189.0,82.0,26.0,101.0,41.0,150.0,False,True
1,1,64,163.0,60.0,8.0,85.0,39.7,34.0,True,False
2,2,51,161.0,64.0,7.0,84.0,39.8,29.0,True,False
3,3,20,192.0,90.0,25.0,105.0,40.7,140.0,False,True
4,4,38,166.0,61.0,25.0,102.0,40.6,146.0,True,False


## Scaling 변환
- 다양한 Scaling (https://scikit-learn.org/stable/api/sklearn.preprocessing.html)
- (실제값 - 예측값)
  + 200-100 / 2-1 => 로그변환을 하는 이유는 오차의 차이를 줄이기 위해

In [10]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import sklearn

# 사이킥런이랑 넘파이 버전 확인
print(sklearn.__version__)
print(np.__version__)

1.6.1
2.2.6


In [11]:
# 각 열마다 단위 다름 == 스케일링 필요
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
train[numeric_features].head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,36,189.0,82.0,26.0,101.0,41.0
1,64,163.0,60.0,8.0,85.0,39.7
2,51,161.0,64.0,7.0,84.0,39.8
3,20,192.0,90.0,25.0,105.0,40.7
4,38,166.0,61.0,25.0,102.0,40.6


In [12]:
# RMSLE 평가지표를 따라가기 위해서 로그변환
# log1p하는 이유는 RMSLE 지표에 log(1+x)가 들어가기 때문 == 더 좋은 평가지표 값 얻기 위해서
y = np.log1p(train['Calories'])  # log1p ==> log(1+x)

# 특성 스케일링
scaler = StandardScaler()
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

X = train[numeric_features]
X_scaled = scaler.fit_transform(X)
X_scaled[0]

array([-0.3571921 ,  1.11523482,  0.49020109,  1.2663241 ,  0.58371421,
        1.23577241])

## 두 변환된 데이터 합치기

In [13]:
X_combined = pd.concat([
    pd.DataFrame(X_scaled, columns = numeric_features),
    train[['Sex_female', 'Sex_male']]
], axis=1)

X_combined.head()

# 고민 거리 예시
# -> Age는 Scaling하는게 맞을까? Duration은? / 구간화가 맞지않을까? 등등

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True
1,1.487943,-0.912137,-1.083172,-0.888309,-1.109436,-0.431163,True,False
2,0.631273,-1.068088,-0.797104,-1.008011,-1.215258,-0.302938,True,False
3,-1.411555,1.349162,1.062337,1.146622,1.007002,0.851095,False,True
4,-0.225397,-0.678209,-1.011655,1.146622,0.689536,0.722869,True,False


# 학습/검증 데이터 분할
- 기본원칙 : 층화추출 (비율에 따라서 데이터셋 분리)
- 아래 코드는 층화추출이 된 상태가 아님 (무작위 샘플링)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_combined, y, test_size = 0.2, random_state = 42  # random_state : 실험 재현성
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((600000, 8), (150000, 8), (600000,), (150000,))

# 모델링
- 다른 알고리즘을 적용
  + 다중회귀(통계 알고리즘)
  + 결정트리
  + 경사하강법(GBM)
  + 결정트리 + 경사하강법 : LightGBM, XGBoost, CatBoost
  + Deep Learning 

In [15]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)  # 모델 학습 끝

DecisionTreeRegressor(random_state=42)

# 모델 평가
- RMSLE 평가 함수 적용
- 실제로는 다양한 모델을 활용해서, 가장 좋은 모델 1개를 선정한다 (필수 과정)
- 평가지표를 확인하는 과정이 반복될거 같으니, 자동화 코드를 만드는 것이 이번주 과제
- 가상) dt_model을 사용하겠음 (가장 좋은 모델로 고려)

In [16]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score # 결정계수

# 평가지표(RMSLE) 함수 생성
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

y_pred = dt_model.predict(X_val)
# print(y_pred, y)

print("검증 데이터 성능:")
print(f"RMSLE: {rmsle(y_val, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_val, y_pred):.2f}")
print(f"R2 Score: {r2_score(y_val, y_pred):.2f}")

검증 데이터 성능:
RMSLE: 0.0246
RMSE: 0.01
R2 Score: 0.99


In [17]:
X_combined.head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True


In [18]:
test.head(1)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8


# 테스트 데이터 예측
- 현재 train에만 Sex관련 데이터 존재 -> 예측으로 test에도 Sex관련 데이터 생성

In [19]:
# 인코딩 확인
test_sex_encoded = pd.get_dummies(test['Sex'], prefix = 'Sex')

# 수치데이터 스케일링
test_numeric = test[numeric_features]
test_scaled = scaler.transform(test_numeric)

# 2개의 데이터를 결합
test_combined = pd.concat([
    pd.DataFrame(test_scaled, columns=test_numeric.columns),
    test_sex_encoded
], axis=1)                

In [20]:
test_combined.head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,0.235887,0.179525,0.418684,-1.008011,-0.897793,-0.302938,False,True
1,-1.016169,1.972969,1.562956,0.548113,0.583714,0.594643,False,True
2,-0.818476,1.037259,0.704752,0.069306,0.689536,0.466417,True,False
3,-0.159499,-0.210354,-0.153452,0.548113,1.218646,0.722869,True,False
4,-0.752578,-0.132378,-0.582554,0.069306,-0.157039,0.594643,True,False


In [21]:
X_combined.head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True
1,1.487943,-0.912137,-1.083172,-0.888309,-1.109436,-0.431163,True,False
2,0.631273,-1.068088,-0.797104,-1.008011,-1.215258,-0.302938,True,False
3,-1.411555,1.349162,1.062337,1.146622,1.007002,0.851095,False,True
4,-0.225397,-0.678209,-1.011655,1.146622,0.689536,0.722869,True,False


In [22]:
from datetime import datetime

# 테스트 데이터 예측
test_pred = dt_model.predict(test_combined)
print(test_pred)

# 위의 test_pred 로그변환의 결과 -> 지수변환해서 스케일로 복원
test_pred = np.exp(test_pred)
print(test_pred)

# 제출 파일 생성 - 공모전 제출 양식에 맞게
submission = pd.DataFrame({
    'id' : test['id'],
    'Calories' : test_pred
})

submission.head()

[3.33220451 4.7095302  4.47733681 ... 4.30406509 5.14166356 4.40671925]
[ 28. 111.  88. ...  74. 171.  82.]


,id,Calories
0,750000,28.0
1,750001,111.0
2,750002,88.0
3,750003,127.0
4,750004,79.0


In [23]:
# 현재 날짜와 시간을 파일명에 포함해서 파일 저장
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
submission.to_csv(f'submission_{current_time}.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import joblib
import time

# 데이터 로드
data = pd.read_csv('train.csv')

# 특성과 타겟 분리
X = data.drop(['id', 'Calories'], axis=1)
y = data['Calories']

# 학습/테스트 데이터 분할 (데이터 누수 방지)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 수치형 특성과 범주형 특성 분리
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
categorical_features = ['Sex']

# 수치형 특성 스케일링 (학습 데이터로만 fit)
scaler = StandardScaler()
X_train_numeric = scaler.fit_transform(X_train[numeric_features])
X_test_numeric = scaler.transform(X_test[numeric_features])

# 범주형 특성 원-핫 인코딩 (학습 데이터로만 fit)
encoder = OneHotEncoder(sparse_output=False)
X_train_categorical = encoder.fit_transform(X_train[categorical_features])
X_test_categorical = encoder.transform(X_test[categorical_features])

# 특성 결합
X_train_processed = np.hstack([X_train_numeric, X_train_categorical])
X_test_processed = np.hstack([X_test_numeric, X_test_categorical])

# 사용할 모델 정의
models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'LinearRegression': LinearRegression(),
    'SVR': SVR()
}

# 각 모델별 하이퍼파라미터 후보군 정의
param_grid = {
    'DecisionTree': [
        {'max_depth': 5},
        {'max_depth': 10}
    ],
    'RandomForest': [
        {'n_estimators': 100},
        {'n_estimators': 200}
    ],
    'LinearRegression': [
        {}
    ],
    'SVR': [
        {'kernel': 'linear'},
        {'kernel': 'rbf'}
    ]
}

# 최고 성능 모델 추적을 위한 변수 초기화
best_score = float('-inf')
best_model_name = None
best_model = None
best_params = None

# 모델 학습 및 평가
for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    for params in param_grid[model_name]:
        # 모델 파라미터 설정
        model.set_params(**params)
        
        # 학습 시간 측정
        start_time = time.time()
        
        # 교차 검증 수행
        cv_scores = cross_val_score(model, X_train_processed, y_train, cv=5, scoring='r2')
        mean_cv = np.mean(cv_scores)
        
        # 모델 학습
        model.fit(X_train_processed, y_train)
        training_time = time.time() - start_time
        
        print(f"Parameters: {params}")
        print(f"Training Time: {training_time:.2f} seconds")
        print(f"Cross-validation R2 Score: {mean_cv:.4f}")
        
        # 최고 성능 모델 업데이트
        if mean_cv > best_score:
            best_score = mean_cv
            best_model_name = model_name
            best_model = model
            best_params = params

# 최고 성능 모델 저장
model_info = {
    'model': best_model,
    'scaler': scaler,
    'encoder': encoder
}
joblib.dump(model_info, 'model.pkl')

# 최종 결과 출력
print("\n=== 최종 결과 ===")
print(f"Best Model: {best_model_name}")
print(f"Best Parameters: {best_params}")
print(f"Best CV Score: {best_score:.4f}")
print("Best model has been saved to 'model.pkl'")

# 파이프라인 구축

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import joblib

# 1. 데이터 로드
data = pd.read_csv('train.csv')

# 2. 특성과 타겟 분리
X = data.drop(['id', 'Calories'], axis=1)
y = data['Calories']

# 3. 수치형 및 범주형 특성
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
categorical_features = ['Sex']

# 4. 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 5. 사용할 모델 정의
models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'LinearRegression': LinearRegression(),
    'SVR': SVR()
}

# 6. 하이퍼파라미터 후보군
param_grid = {
    'DecisionTree': [{'max_depth': 5}, {'max_depth': 10}],
    'RandomForest': [{'n_estimators': 100}, {'n_estimators': 200}],
    'LinearRegression': [{}],
    'SVR': [{'kernel': 'linear'}, {'kernel': 'rbf'}]
}

# 7. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. 최적 모델 탐색
best_score = float('-inf')
best_model_name = None
best_pipeline = None
best_params = None

for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    for params in param_grid[model_name]:
        # 파라미터 세팅
        model.set_params(**params)

        # 전체 파이프라인 구성
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])

        # 교차 검증
        start_time = time.time()
        scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
        elapsed = time.time() - start_time
        mean_score = np.mean(scores)